In [ ]:
import pandas as pd
from openpyxl import load_workbook


In [ ]:
df_medlemmar =  pd.read_excel(r"G:\Min enhet\Styrelsens dokument\Träning\Gradering.xlsx", sheet_name="Medlemmar")
df_medlemmar = df_medlemmar.drop(columns=['Skapad', 'Född', 'Startade', 'Tränat (år)' ])
df_medlemmar = df_medlemmar[df_medlemmar['Aktiv'] == 'Ja']
df_medlemmar['Aktiv'] = True
df_medlemmar['Ålder'] = pd.to_numeric(df_medlemmar['Ålder'], errors='coerce').fillna(0)

In [ ]:
df_tekniker =  pd.read_excel(r"G:\Min enhet\Styrelsens dokument\Träning\Gradering.xlsx", sheet_name="Tekniker")
df_tekniker = df_tekniker[df_tekniker['Version'] == 'ÄJK 1.0']

In [ ]:
df_graderingsregler =  pd.read_excel(r"G:\Min enhet\Styrelsens dokument\Träning\Gradering.xlsx", sheet_name="Graderingsregler")
df_graderingsregler = df_graderingsregler.drop(columns=['Ålder', 'Träningspass', '0'])
df_graderingsregler['Ålder2'] = pd.to_numeric(df_graderingsregler['Ålder2'], errors='coerce').astype(float)

In [ ]:
age_rules = df_graderingsregler[['Ålder2', 'Dec']].sort_values('Ålder2')
tmp = pd.merge_asof( df_medlemmar.reset_index().sort_values('Ålder'), age_rules, left_on='Ålder', right_on='Ålder2', direction='backward')
df_medlemmar['Borde_ålder'] = tmp.sort_values('index').set_index('index')['Dec'].values

In [ ]:
traningspass_rules = df_graderingsregler[['Akumelerat', 'Dec']].sort_values('Akumelerat')
tmp = pd.merge_asof( df_medlemmar.reset_index().sort_values('Närvaro'), traningspass_rules, left_on='Närvaro', right_on='Akumelerat', direction='backward')
df_medlemmar['Borde_närvaro'] = tmp.sort_values('index').set_index('index')['Dec'].values

df_medlemmar

In [ ]:
df_medlemmar['Borde'] = df_medlemmar[['Borde_ålder', 'Borde_närvaro']].max(axis=1)
df_medlemmar = df_medlemmar.drop(columns=['Borde_ålder', 'Borde_närvaro'])
df_medlemmar[df_medlemmar['ID'] == "BosseAlmén"]

In [ ]:
kyu_to_dec = df_graderingsregler.set_index('Kyu')['Dec']
kyu_to_dec

In [ ]:


df_medlemmar['Grad_Dec'] = df_medlemmar['Grad'].map(kyu_to_dec)
df_medlemmar['Diff'] = df_medlemmar['Grad_Dec']-df_medlemmar['Borde']
df_medlemmar = df_medlemmar.sort_values('Diff')

df_medlemmar

In [ ]:
dec_to_kyu = df_graderingsregler.set_index('Dec')['Kyu']
df_medlemmar['Borde'] = df_medlemmar['Borde'].map(dec_to_kyu)
df_medlemmar = df_medlemmar.drop(columns=['Grad_Dec','Förnamn', 'Efternamn'])
df_medlemmar = df_medlemmar.rename(columns={'ID': 'email'})
df_medlemmar = df_medlemmar.rename(columns={'Närvaro': 'Attendance'})
df_medlemmar = df_medlemmar.rename(columns={'Grad': 'Grade'})
df_medlemmar = df_medlemmar.rename(columns={'Aktiv': 'Active'})
df_medlemmar = df_medlemmar.rename(columns={'Ålder': 'Age'})




df_medlemmar["password"] = "b252bdb8"
df_medlemmar["Club"] = "Ängelholms JK"


df_medlemmar

In [ ]:
df_medlemmar.columns = df_medlemmar.columns.str.lower()
df_medlemmar.to_json(r"..\judotech.web.club\source\data\medlemmar.json", orient="records", force_ascii=False, indent=2)
